In [ ]:
import pandas as pd
from collections import defaultdict
from itertools import combinations
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load CSV files into DataFrames
aisles = pd.read_csv('/content/drive/My Drive/instacart-market-basket-analysis/aisles.csv')
departments = pd.read_csv('/content/drive/My Drive/instacart-market-basket-analysis/departments.csv')
order_products_prior = pd.read_csv('/content/drive/My Drive/instacart-market-basket-analysis/order_products__prior.csv')
order_products_train = pd.read_csv('/content/drive/My Drive/instacart-market-basket-analysis/order_products__train.csv')
orders = pd.read_csv('/content/drive/My Drive/instacart-market-basket-analysis/orders.csv')
products = pd.read_csv('/content/drive/My Drive/instacart-market-basket-analysis/products.csv')

The following code defines a function called reduce_memory_usage that aims to optimize the memory usage of a pandas DataFrame by converting its columns to more memory-efficient data types. The function iterates over each column in the DataFrame and checks its data type. If the column is not of type object (typically used for strings), it determines the minimum and maximum values in the column. Based on these values, it converts the column to the smallest possible integer or float type that can accommodate the range of values, thereby reducing memory usage. For example, if the column contains integer values that fit within the range of an int8, it converts the column to int8. Similarly, for floating-point numbers, it converts the column to float32 if the values fit within the range of a float32. After defining this function, the code applies it to several DataFrames (order_products_prior, order_products_train, orders, and products) to optimize their memory usage. This is particularly useful when working with large datasets, as it helps to reduce the overall memory footprint and improve performance.

In [ ]:
# Function to reduce memory usage
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > -128 and c_max < 127:
                    df[col] = df[col].astype('int8')
                elif c_min > -32768 and c_max < 32767:
                    df[col] = df[col].astype('int16')
                elif c_min > -2147483648 and c_max < 2147483647:
                    df[col] = df[col].astype('int32')
                else:
                    df[col] = df[col].astype('int64')
            else:
                if c_min > -3.4e38 and c_max < 3.4e38:
                    df[col] = df[col].astype('float32')
                else:
                    df[col] = df[col].astype('float64')
    return df

# Reduce memory usage
order_products_prior = reduce_memory_usage(order_products_prior)
order_products_train = reduce_memory_usage(order_products_train)
orders = reduce_memory_usage(orders)
products = reduce_memory_usage(products)

The following code concatenates the order_products__prior and order_products__train DataFrames into a single DataFrame called order_products. It then performs a series of merges to enrich this DataFrame with additional details. First, it merges order_products with the orders DataFrame to include user and order details. Next, it merges the resulting DataFrame with the products DataFrame to add product details. Subsequently, it merges with the aisles DataFrame to incorporate aisle information, and finally, it merges with the departments DataFrame to include department details. After these merges, the code sorts the order_products DataFrame by user_id and order_number to ensure that the orders are processed in the correct sequence for each user.

In [ ]:
# Concatenate order_products__prior and order_products__train
order_products = pd.concat([order_products_prior, order_products_train])

# Merge order_products with orders to get user and order details
order_products = order_products.merge(orders, on='order_id', how='left')

# Merge the resulting DataFrame with products to get product details
order_products = order_products.merge(products, on='product_id', how='left')

# Merge the resulting DataFrame with aisles to get aisle details
order_products = order_products.merge(aisles, on='aisle_id', how='left')

# Merge the resulting DataFrame with departments to get department details
order_products = order_products.merge(departments, on='department_id', how='left')

# Sort and group the merged data to ensure that you process the orders in the correct sequence for each user
order_products = order_products.sort_values(by=['user_id', 'order_number'])

The following code performs several data preprocessing steps to prepare the dataset for analysis. First, it filters out users with very few transactions. It sets a minimum threshold of 5 transactions (min_transactions = 5) and calculates the number of transactions for each user using the value_counts method on the user_id column. It then identifies the users who meet or exceed this threshold and filters the order_products DataFrame to include only these valid users.

Next, the code filters out items that appear infrequently. It sets a minimum item frequency threshold of 5 (min_item_frequency = 5) and calculates the number of occurrences for each product using the value_counts method on the product_id column. It identifies the products that meet or exceed this threshold and filters the order_products DataFrame to include only these valid items.

To ensure that each transaction is unique for each user, the code removes duplicate entries by dropping rows with the same combination of user_id, order_id, and product_id using the drop_duplicates method.

Finally, the code handles missing values by dropping any rows that contain missing values using the dropna method. These preprocessing steps help to clean and refine the dataset, ensuring that it is suitable for subsequent analysis.

In [ ]:
# Filter out users with very few transactions
min_transactions = 5
user_order_counts = order_products['user_id'].value_counts()
valid_users = user_order_counts[user_order_counts >= min_transactions].index
order_products = order_products[order_products['user_id'].isin(valid_users)]

# Filter out items that appear infrequently
min_item_frequency = 5
item_order_counts = order_products['product_id'].value_counts()
valid_items = item_order_counts[item_order_counts >= min_item_frequency].index
order_products = order_products[order_products['product_id'].isin(valid_items)]

# Ensure each transaction is unique for each user by removing duplicates
order_products = order_products.drop_duplicates(subset=['user_id', 'order_id', 'product_id'])

# Handle missing values
order_products = order_products.dropna()

The following code defines and implements the SPADE (Sequential Pattern Discovery using Equivalence classes) algorithm to discover frequent sequential patterns in transaction data.

First, the generate_candidates function is defined to generate candidate sequences of a specified length. It takes the current frequent sequences and valid items, and creates new sequences by appending each valid item to the existing sequences, ensuring that the same item is not repeatedly added.

Next, the count_support function is defined to count the support of each candidate sequence. It iterates through each candidate and each transaction, checking if the candidate sequence is present in the transaction. It maintains a count of how many transactions contain each candidate sequence. Progress is printed every 100 candidates or when the count reaches the total number of candidates.

The prune_candidates function is then defined to prune candidate sequences that do not meet the minimum support threshold. It returns a dictionary of sequences that have support counts greater than or equal to the specified minimum support.

The spade function implements the SPADE algorithm. It initializes an empty dictionary sequences to store frequent sequences and their support counts. It starts with sequences of length 1 and iteratively generates candidates of increasing lengths. For each length, it generates candidates, counts their support, prunes infrequent candidates, and updates the sequences dictionary with the pruned candidates. The process continues until no more candidates can be generated. The function returns a sorted list of frequent sequences and their support counts.

The code then prepares the transactions for the SPADE algorithm by grouping the order_products DataFrame by user_id and creating a list of product IDs for each user.

A minimum support threshold is defined as 20% of the total number of transactions.

Finally, the SPADE algorithm is run with the prepared transactions and the minimum support threshold. The frequent patterns discovered by the algorithm are printed, excluding the empty sequence.

This implementation helps to identify frequent sequential patterns in the transaction data, which can be useful for various analytical purposes, such as market basket analysis and recommendation systems.

In [ ]:
# Function to generate candidate sequences
def generate_candidates(sequences, length):
    candidates = set()
    for seq in sequences:
        for item in valid_items:
            if item not in seq:  # Ensure the same item is not repeatedly added
                new_seq = seq + (item,)
                if len(new_seq) == length:
                    candidates.add(new_seq)
    return candidates

# Function to count support of each candidate sequence
def count_support(candidates, transactions):
    support_counts = defaultdict(int)
    num_candidates = len(candidates)
    for i, candidate in enumerate(candidates, 1):
        for transaction in transactions:
            if all(item in transaction for item in candidate):
                support_counts[candidate] += 1
        if i % 100 == 0 or i == num_candidates:
            print(f"Progress: {i}/{num_candidates}")
    return support_counts

# Function to prune candidates that do not meet the minimum support threshold
def prune_candidates(support_counts, min_support):
    return {seq: count for seq, count in support_counts.items() if count >= min_support}

# Function to implement the SPADE algorithm
def spade(transactions, min_support):
    sequences = {(): len(transactions)}
    length = 1
    while True:
        print(f"Generating candidates of length {length}...")
        candidates = generate_candidates(sequences.keys(), length)
        print(f"Number of candidates: {len(candidates)}")

        print("Counting support for candidates...")
        support_counts = count_support(candidates, transactions)
        total_support = sum(support_counts.values())
        print(f"Total support count for length {length}: {total_support}")

        print("Pruning candidates...")
        pruned_candidates = prune_candidates(support_counts, min_support)
        print(f"Pruned candidates: {pruned_candidates}")

        if not pruned_candidates:
            break
        sequences.update(pruned_candidates)
        length += 1
    return sorted(sequences.items(), key=lambda x: x[1], reverse=True)

# Prepare transactions for SPADE algorithm
transactions = order_products.groupby('user_id')['product_id'].apply(list).tolist()

# Define minimum support threshold
min_support = 0.2 * len(transactions)  # Adjusted to a lower value

# Run SPADE algorithm
frequent_patterns = spade(transactions, min_support)

# Output the frequent patterns, excluding the empty sequence
frequent_patterns = [pattern for pattern in frequent_patterns if pattern[0]]
print(frequent_patterns)

Generating candidates of length 1...
Number of candidates: 47975
Counting support for candidates...
Progress: 100/47975
Progress: 200/47975


KeyboardInterrupt: 